In [1]:
import pandas as pd
import numpy as np
import importlib

import experiment_runner
import generate_synthetic_data
from experiment_runner import ExperimentRunner
from generate_synthetic_data import GenerateSyntheticData
importlib.reload(experiment_runner)
importlib.reload(generate_synthetic_data)
from sklearn.model_selection import train_test_split

C:\Users\sidrt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Steps
1. Import the files
2. Transform all their features
3. Use the experiment runner and the generate synthetic data class to get info for the experiments


In [ ]:
API_KEY = ""

In [ ]:
# Load dataset (replace with your dataset)
dataset = pd.read_csv('car.csv') 
target_column = 'Transmission' 
#subset_sizes = [0.01, 0.1, 0.3, 0.5, 0.7, 1.0]  # Proportions of the dataset

subset_sizes = [0.01]

# Initialize synthetic data generator and experiment runner
synthetic_data_generator = GenerateSyntheticData(API_KEY)
experiment_runner = ExperimentRunner(dataset, target_column)

# Loop through subset sizes
results = []
for prop in subset_sizes:
    # Take subset of the data
    subset = dataset.sample(frac=prop, random_state=42)
    
    # Compute subset characteristics (dimensions, variance, skewness)
    characteristics = experiment_runner.compute_characteristics(subset)
    
    # Generate synthetic data using subset
    synthetic_data = synthetic_data_generator.predict(10,subset)
    print(synthetic_data)
    
    # Combine real and synthetic data for training
    # combined_data = pd.concat([subset, synthetic_data])
    combined_data = subset

    # Split combined data into train/test
    train_data, test_data = train_test_split(combined_data, test_size=0.2, random_state=42)
    
    # Train neural network
    experiment_runner.train_network(train_data)
    
    # Benchmark network
    mse = experiment_runner.benchmark_network(test_data)
    
    # Save results
    results.append({
        "subset_proportion": prop,
        "characteristics": characteristics,
        "mse": mse
    })

# Print results
for result in results:
    print(f"Subset Proportion: {result['subset_proportion']}")
    print(f"Characteristics: {result['characteristics']}")
    print(f"Mean Squared Error: {result['mse']}\n")


c:\siddharth\programming\school\am226-final-proj\experiment_runner.py:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  characteristics['mean'] = full_data.mean().to_dict()
c:\siddharth\programming\school\am226-final-proj\experiment_runner.py:27: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  characteristics['std'] = full_data.std().to_dict()
c:\siddharth\programming\school\am226-final-proj\experiment_runner.py:30: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  characteristics['median'] 

```json
[
  {"Car_Name": "Maruti Suzuki Swift", "Year": 2019, "Selling_Price": 4.5, "Present_Price": 6.8, "Kms_Driven": 35000, "Fuel_Type": "Petrol", "Seller_Type": "Dealer", "Transmission": "Manual", "Owner": 0},
  {"Car_Name": "Hyundai Creta", "Year": 2017, "Selling_Price": 8.75, "Present_Price": 12.5, "Kms_Driven": 42000, "Fuel_Type": "Diesel", "Seller_Type": "Dealer", "Transmission": "Automatic", "Owner": 1},
  {"Car_Name": "Honda City", "Year": 2015, "Selling_Price": 3.2, "Present_Price": 5.0, "Kms_Driven": 60000, "Fuel_Type": "Petrol", "Seller_Type": "Individual", "Transmission": "Manual", "Owner": 0},
  {"Car_Name": "Mahindra XUV500", "Year": 2018, "Selling_Price": 10.0, "Present_Price": 14.0, "Kms_Driven": 55000, "Fuel_Type": "Diesel", "Seller_Type": "Dealer", "Transmission": "Manual", "Owner": 0},
  {"Car_Name": "Tata Nexon", "Year": 2020, "Selling_Price": 7.0, "Present_Price": 9.5, "Kms_Driven": 28000, "Fuel_Type": "Petrol", "Seller_Type": "Dealer", "Transmission": "Manual", 

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.